# Gaussian Mixture Modeling

In [ ]:
# importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn import metrics

from sklearn.utils.extmath import row_norms
from sklearn.datasets._samples_generator import make_blobs
from timeit import default_timer as timer
import sys
numpy.set_printoptions(threshold=sys.maxsize)

The required input are the first two dimensions of the MDS matrix obtained from the Hamming distance matrix:

In [ ]:
df = pd.read_csv('mds.df.csv')

First let's find the optimal number of clusters/components:

In [ ]:
data = np.array(df[['x','y']])

n_components = np.arange(1, 10)
models = [GaussianMixture(n, covariance_type='full', random_state=0).fit(data)
          for n in n_components]
plt.plot(n_components, [m.bic(data) for m in models], label='BIC')
plt.plot(n_components, [m.aic(data) for m in models], label='AIC')

allbics = [m.bic(data) for m in models]
allaics = [m.aic(data) for m in models]
minbicind = allbics.index(min([m.bic(data) for m in models]))
minaicind = allaics.index(min([m.aic(data) for m in models]))

plt.axvline(x=n_components[minbicind],c=(0,0,0),linestyle='--')
# plt.text((n_components[minbicind] + 0.1),-2000,'min(BIC)',rotation=90)
plt.axvline(x=n_components[minaicind],c=(0,0,0),linestyle='--')
# plt.text((n_components[minaicind] + 0.1),-2000,'min(AIC)',rotation=90)

plt.legend(loc='best')
plt.xlabel('n_components')
plt.ylabel('AIC/BIC score')
plt.savefig('aicbic.jpeg', dpi=300)
plt.show()

S=[]

# Range of clusters to try (2 to 10)
K=range(2,10)

# Select data for clustering model
X = data

for k in K:
    # Set the model and its parameters
    model = GaussianMixture(n_components=k, n_init=20, init_params='kmeans', random_state=1234)
    # Fit the model 
    labels = model.fit_predict(X)
    # Calculate Silhoutte Score and append to a list
    S.append(metrics.silhouette_score(X, labels, metric='euclidean'))

maxsil = S.index(max(S))
print(maxsil)
print(S)

# Plot the resulting Silhouette scores on a graph
plt.plot(K, S, 'bo-', c=(0,0,0))
plt.axvline(x=K[maxsil],c=(0,0,0),linestyle='--')
plt.xlabel('k')
plt.ylabel('Silhouette Score')
plt.savefig('silhouette.jpeg', dpi=300)
plt.show()

We'll take the optimal number of clusters and generate a plot:

In [ ]:
clus = 4

def get_initial_means(X, init_params, r):
        # Run a GaussianMixture with max_iter=0 to output the initalization means
        gmm = GaussianMixture(
            n_components=clus, init_params=init_params, tol=1e-9, max_iter=2000, random_state=r
        ).fit(X)
        return gmm.means_
    
r = np.random.RandomState(seed=1234)

plt.plot()

start = timer()
ini = get_initial_means(X, 'kmeans', r)
end = timer()
init_time = end - start

gmm = GaussianMixture(
    n_components=clus, means_init=ini, tol=1e-9, max_iter=2000, random_state=r).fit(X)

labels = gmm.predict(X)
print(labels)

for i, color in enumerate(colors):
    data = X[gmm.predict(X) == i]
    df2 = pd.DataFrame(data)
    df2.to_csv(f'{str(i)}.csv')
    plt.scatter(data[:, 0], data[:, 1], color=color, marker="x")

plt.xticks(())
plt.yticks(())
plt.savefig('gmm.jpeg',dpi=300)
plt.show()

Or, you can choose 4 different numbers of clusters and plot the resulting GMM models side by side to compare:

In [ ]:
nclus = [3,4,5,6]

logliks = pd.DataFrame()

X = np.array(df[['x','y']])

colors = ["navy", "turquoise", "cornflowerblue", "darkorange","darkgreen","brown"]
times_init = {}
relative_times = {}


plt.figure(figsize=(4 * len(nclus) // 2, 6))
plt.subplots_adjust(
    bottom=0.1, top=0.9, hspace=0.15, wspace=0.05, left=0.05, right=0.95
)

for n, clus in enumerate(nclus):
    def get_initial_means(X, init_params, r):
        # Run a GaussianMixture with max_iter=0 to output the initalization means
        gmm = GaussianMixture(
            n_components=clus, init_params=init_params, tol=1e-9, max_iter=2000, random_state=r
        ).fit(X)
        return gmm.means_
    
    r = np.random.RandomState(seed=1234)
    plt.subplot(2, len(nclus) // 2, n + 1)

    start = timer()
    ini = get_initial_means(X, 'kmeans', r)
    end = timer()
    init_time = end - start

    gmm = GaussianMixture(
        n_components=clus, means_init=ini, tol=1e-9, max_iter=2000, random_state=r).fit(X)
    
    labels = gmm.predict(X)
    print(labels)

    for i, color in enumerate(colors):
        data = X[gmm.predict(X) == i]
        df2 = pd.DataFrame(data)
        df2.to_csv(f'{str(i)}.csv')
        plt.scatter(data[:, 0], data[:, 1], color=color, marker="x")

    plt.xticks(())
    plt.yticks(())
plt.savefig('gmm_compare.jpeg',dpi=300)
plt.show()